<a name='1'></a>
## Prerequisites

In [ ]:
%pip install -q "openvino>=2023.1.0"
!git clone https://github.com/svc-develop-team/so-vits-svc -b 4.1-Stable
%cd so-vits-svc
%pip install --upgrade pip setuptools
%pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
!wget https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -O pretrain/checkpoint_best_legacy_500.pt
!wget "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Rainbow%20Dash%20(singing)/kmeans_10000.pt" -O logs/44k/kmeans_10000.pt
!wget "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Rainbow%20Dash%20(singing)/config.json" -O configs/config.json
!wget "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Rainbow%20Dash%20(singing)/G_30400.pth" -O logs/44k/G_30400.pth
!wget "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Rainbow%20Dash%20(singing)/D_30400.pth" -O logs/44k/D_30400.pth
!wget "https://huggingface.co/datasets/santifiorino/spinetta/resolve/main/spinetta/000.wav" -O raw/000.wav

## Use the original model to run an inference

In [1]:
# %cd so-vits-svc

/mnt/c/Users/amokrov/PycharmProjects/default/my_openvino_notebooks/notebooks/259-sofvc-voice-conversion/so-vits-svc


In [2]:
from inference.infer_tool import Svc

model = Svc("logs/44k/G_30400.pth", "configs/config.json", device='cpu')

/home/amokrov/.virtualenvs/my_openvino_notebooks/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


load 


2023-10-11 18:24:01.873828: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-11 18:24:02.234937: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-11 18:24:04.713462: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.1.0+cu118)
    Python  3.8.16 (you have 3.8.10)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
load model(s) from pretrain/checkpoint_best_legacy_500.pt


/home/amokrov/.virtualenvs/my_openvino_notebooks/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
kwarg = {
    'raw_audio_path': 'raw/000.wav', 
    'spk': 'Rainbow Dash (singing)', 
    'tran': 0, 
    'slice_db': -40, 
    'cluster_infer_ratio': 0, 
    'auto_predict_f0': False, 
    'noice_scale': 0.4, 
    'pad_seconds': 0.5, 
    'clip_seconds': 0, 
    'lg_num': 0, 
    'lgr_num': 0.75, 
    'f0_predictor': 'pm', 
    'enhancer_adaptive_key': 0, 
    'cr_threshold': 0.05, 
    'k_step': 100, 
    'use_spk_mix': False, 
    'second_encoding': False, 
    'loudness_envelope_adjustment': 1
}

audio = model.slice_inference(**kwarg)

#=====segment start, 8.44s======


/mnt/c/Users/amokrov/PycharmProjects/default/my_openvino_notebooks/notebooks/259-sofvc-voice-conversion/so-vits-svc/inference/infer_tool.py:270: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


vits use time:5.225666522979736
#=====segment start, 4.62s======
vits use time:3.4846408367156982
#=====segment start, 0.002s======
jump empty segment


In [ ]:
import IPython.display as ipd

ipd.Audio(audio, rate=model.target_sample)

## Convert to OpenVINO IR model

In [ ]:
import openvino as ov
import torch
from pathlib import Path


dummy_c = torch.randn(1, 256, 813)
dummy_f0 = torch.randn(1, 813)
dummy_uv = torch.ones(1, 813)
dummy_g = torch.tensor([[0]])
model.net_g_ms.forward = model.net_g_ms.infer

#input_info = [("c", dummy_c.shape, torch.float32),("f0", dummy_f0.shape, torch.float32),("uv", dummy_uv.shape, torch.float32), ("g", dummy_g.shape, torch.int64)]
net_g_kwargs = {
    'c': dummy_c,
    'f0': dummy_f0,
    'uv': dummy_uv,
    'g': dummy_g,
    
    # 'noice_scale': 0.35,
    # 'seed': 52468,
    # 'predict_f0': False,
    # 'vol':  0
}
core = ov.Core()

converted_model = ov.convert_model(model.net_g_ms, example_input=net_g_kwargs)

net_g_model_xml_path = Path('models/ov_net_g_model.xml')

net_g_model_xml_path.parent.mkdir(parents=True, exist_ok=True)
ov.save_model(converted_model, net_g_model_xml_path)
#compiled_net_g_model = core.compile_model(net_g_model_xml_path)

In [ ]:
import ipywidgets as widgets
import openvino as ov

core = ov.Core()

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

In [ ]:
class NetGModelWrapper:
    def __init__(self, net_g_model_xml_path):
        super().__init__()
        self.net_g_model = core.compile_model(net_g_model_xml_path, device.value)
        
    def infer(self, c, *, f0, uv, g, noice_scale=0.35, seed=52468, predict_f0=False, vol = None):
        print(self.net_g_model)
        print(c.shape, f0.shape, uv.shape, g.shape)
        #results = self.net_g_model({'c': c, 'f0': f0, 'uv': uv, 'g': g})[0]
        results = self.net_g_model((c, f0, uv, g))[0]
        return results

        
#compiled_net_g_model.create_infer_request()
model.net_g_ms = NetGModelWrapper(net_g_model_xml_path)
#compiled_net_g_model.infer = compiled_net_g_model.__call__
audio = model.slice_inference(**kwarg)

## Convert model by parts

In [ ]:
model = Svc("logs/44k/G_30400.pth", "configs/config.json", device='cpu')

In [5]:
# x.shape=torch.Size([1, 192, 813]), x_mask.shape=torch.Size([1, 1, 813]), f0_to_coarse(f0).shape=torch.Size([1, 813]), noice_scale=0.4

import openvino as ov
import torch
from pathlib import Path
from utils import f0_to_coarse


dummy_x = torch.randn(1, 192, 813)
dummy_x_mask = torch.randn(1, 1, 813)
f0 = torch.randn(1, 813)
noice_scale = torch.tensor(0.4)

core = ov.Core()

#converted_enc_p_model = ov.convert_model(model.net_g_ms.enc_p, example_input=(dummy_x, dummy_x_mask, f0_to_coarse(f0), noice_scale))
converted_enc_p_model = ov.convert_model(model.net_g_ms.enc_p.enc_, example_input=(dummy_x, dummy_x_mask))

enc_p_model_xml_path = Path('models/ov_enc_p_model.xml')

enc_p_model_xml_path.parent.mkdir(parents=True, exist_ok=True)
ov.save_model(converted_enc_p_model, enc_p_model_xml_path)
#compiled_enc_p_model = core.compile_model(enc_p_model_xml_path)

/mnt/c/Users/amokrov/PycharmProjects/default/my_openvino_notebooks/notebooks/259-sofvc-voice-conversion/so-vits-svc/modules/attentions.py:217: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert t_s == t_t, "Relative attention is only available for self-attention."
/mnt/c/Users/amokrov/PycharmProjects/default/my_openvino_notebooks/notebooks/259-sofvc-voice-conversion/so-vits-svc/modules/attentions.py:262: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  pad_length = max(length - (self.window_size + 1), 0)
/mnt/c/Users/amokrov/PycharmProjects/default/my_o

In [6]:
compiled_enc_p_model = core.compile_model(enc_p_model_xml_path)
compiled_enc_p_model((dummy_x, dummy_x_mask))

RuntimeError: Exception from src/inference/src/infer_request.cpp:224:
[ PARAMETER_MISMATCH ] Can not clone with new dims. Descriptor's shape: {1, ? - ?, 96} is incompatible with provided dimensions: {1, 1625, 96}.


In [ ]:
# z_p.shape=torch.Size([1, 192, 813]), c_mask.shape=torch.Size([1, 1, 813]), g.shape=torch.Size([1, 256, 1])
dummy_z_p = torch.randn(1, 192, 813)
dummy_c_mask = torch.randn(1, 1, 813)
dummy_g = torch.randn(1, 256, 1)
dummy_reverse = torch.tensor(True)

converted_flow_model = ov.convert_model(model.net_g_ms.flow, example_input=(dummy_z_p, dummy_c_mask, dummy_g, dummy_reverse))

flow_model_xml_path = Path('models/ov_flow_model.xml')

flow_model_xml_path.parent.mkdir(parents=True, exist_ok=True)
ov.save_model(converted_flow_model, flow_model_xml_path)
#compiled_flow_model = core.compile_model(flow_model_xml_path)

In [ ]:
# (z * c_mask).shape=torch.Size([1, 192, 813]), g.shape=torch.Size([1, 256, 1]), f0.shape=torch.Size([1, 813])

dummy_z_c_mask = torch.randn(1, 192, 813)
dummy_g = torch.randn(1, 256, 1)
f0 = torch.randn(1, 813)


converted_dec_model = ov.convert_model(model.net_g_ms.dec, example_input=(dummy_z_c_mask, f0, dummy_g))

dec_model_xml_path = Path('models/ov_dec_model.xml')

dec_model_xml_path.parent.mkdir(parents=True, exist_ok=True)
ov.save_model(converted_dec_model, dec_model_xml_path)
#compiled_dec_model = core.compile_model(dec_model_xml_path)

In [ ]:
model = Svc("logs/44k/G_30400.pth", "configs/config.json", device='cpu')

class EncPModelWrapper(torch.nn.Module):
    def __init__(self, model_path):
        super().__init__()
        self.compiled_model = core.compile_model(model_path, device.value)

    # def forward(self, x, x_mask, f0=None, noice_scale=1):
    #     return self.compiled_model((x, x_mask, f0, noice_scale))[0]

    def forward(self, x, x_mask):
        x = self.compiled_model((x, x_mask))[0]
        return x


class FlowWrapper(torch.nn.Module):
    def __init__(self, model_path):
        super().__init__()
        self.compiled_model = core.compile_model(model_path, device.value)

    def forward(self, x, x_mask, g=None, reverse=False):
        z = self.compiled_model((x, x_mask, g, reverse))[0]
        return torch.tensor(z)


class DecWrapper(torch.nn.Module):
    def __init__(self, model_path):
        super().__init__()
        self.compiled_model = core.compile_model(model_path, device.value)

    def forward(self, z_c_mask, *, g, f0):
        o = self.compiled_model((z_c_mask, f0, g))[0]
        return torch.tensor(o)


# model.net_g_ms.enc_p.enc_ = EncPModelWrapper(enc_p_model_xml_path)
model.net_g_ms.flow = FlowWrapper(flow_model_xml_path)
model.net_g_ms.dec = DecWrapper(dec_model_xml_path)

In [ ]:
model.net_g_ms.enc_p.enc_.compiled_model((dummy_x, dummy_x_mask))

In [ ]:
audio = model.slice_inference(**kwarg)

In [ ]:
import IPython.display as ipd

ipd.Audio(audio, rate=model.target_sample)